# Decomposing InSAR velocities from multiple lines-of-sight

Gareth Funning, University of California, Riverside

InSAR displacements and velocities are one-dimensional measurements $-$ the three-dimensional movement of the ground surface is projected into the radar line-of-sight (LOS). While many of us develop some intuition for how surface deformation is represented in LOS, it is still often more intuitive to consider deformation in directions that come more naturally to us, such as the vertical direction, or horizontal directions. Luckily, if we have enough information $-$ for example, from multiple different lines-of-sight $-$ we can decompose the LOS information into the directions of our choice... 


## 0. Dependencies

We don't need many libraries to do this, but we do need numpy...

In [ ]:
import numpy as np       # what is life without numpy?

## 1. The Berkeley landslides

The example we will use here is of the Berkeley Hills landslides in northern California. For several decades, it has been apparent, from damage to roads, pipes and houses, that there are multiple creeping landslides in the hills to the northwest of the University of California campus in Berkeley, where many of the faculty, staff and students live. (I lived in that area as a postdoc, and discovered, when I saw the InSAR data for the area, that I was living on one of them! Oops.) These landslides are fast-moving compared with the other hazardous geologic structures in the area, such as the Hayward fault, and are prominent in InSAR data from the area. 

We will look at persistent scatterer InSAR data (the image below) from two different viewing geometries that capture the landslide signals well $-$ an ERS descending track data set (top), and a Radarsat ascending track dataset (bottom). 

Some useful details:

- Both data sets show **range change** velocity, estimated **with respect to 'stable Berkeley’** (the SW corner), **whose velocity is set to zero**. 
- **Range change** is the sign convention in which **positive velocities indicate the ground is moving away from the satellite (and *vice-versa*)**.
- Although the data sets cover different (but overlapping) time periods, we are going to assume that the **average velocities are constant over the whole time period**, and thus can be directly compared (and combined). 
- Track (T) and 'Look' (L) directions are plotted as white arrows on each plot. Note that **both satellites were right-looking**.
- The flight azimuth ($\phi$) and incidence angles ($\theta$) for each data set are provided
- **The five largest landslides appear as small, self-contained areas**, outlined in pink, **whose LOS velocities differ from their surrounding areas**. The average velocity within each area is provided on the plot.

![plots of two InSAR datasets](images/berkeley_landslides.png)

At first glance, there are some obvious differences between the plots of the descending and ascending data: 

- **The sense of relative motion between the landslides and their surrounding areas is different.** (What does that imply about the motion of the landslides $-$ are they moving primarily vertically or horizontally? What is your reasoning there?)

- You may also notice that the **descending track data show an abrupt change in LOS velocity in the easternmost area of the data set**. This is the signature of creep (slow, aseismic movement) on the Hayward fault, a strike-slip fault responsible for a M~7 earthquake in 1868. The surface fault trace is located along the change in velocity. (Assuming that all of the fault-related deformation is horizontal, what is the sense of motion of the fault $-$ left-lateral or right-lateral?)

## 2. Line-of-sight vectors: a review

We have met LOS vectors before, when considering the **forward problem** of estimating what the LOS displacement would be for a given three-dimensional displacement. Now, of course, we want to solve the **inverse problem**, of what the displacement in a particular direction should be, given the LOS deformation in two or more different viewing geometries.

The estimation of the LOS vector is purely geometric, and relates to the direction the radar is pointing, and the incidence angle of the radar with the ground. All of the major SAR satellite missions to date have been right-looking $-$ their radars point 90$^\circ$ to the right of their flight azimuth. (**NB** NISAR will be the first major satellite mission to have a permanently *left-looking* radar antenna, which will be very exciting, and will help to constrain diplacements in 3D when combined with right-looking data from Sentinel-1.) 

We are going to make a couple of simplifying assumptions when calculating the LOS vectors for our data:
- The pointing direction of the radar is fixed (which is appropriate for stripmap data)
- The incidence angle of the radar is fixed (which is reasonable for a small area)

And the rest is just geometry and trigonometry!

First, we can look at the incidence of the radar (represented by the unit vector $\hat{p}$) in cross-section, and decompose it into horizontal and vertical components:

![diagram in cross-section view of horizontal and vertical components of incidence](images/vertical.png)

(The negative sign here is due to the downward pointing of the radar.)

Next, we consider the horizontal component of incidence, which is itself a function of the east and north components. 

![diagram in map view decomposing the horizontal component into north and east directions](images/horizontal.png)

With this, we have all three components of the LOS vector, $\hat{p}$, defined:

$$\hat{p} = [p_x, p_y, p_z] = [\cos\phi\sin\theta, -\sin\phi\sin\theta, -\cos\theta]$$

Let's calculate them for our two data sets, using the values of (flight) azimuth and incidence for each one!

In [ ]:
# calculate the LOS vector components for the ascending data (you need to enter the values)

asc_azi=np.radians()            # remember that Python does its trig in radians

asc_inc=np.radians()

p_asc=np.array([np.cos(asc_azi)*np.sin(asc_inc),-np.sin(asc_azi)*np.sin(asc_inc),-np.cos(asc_inc) ])

In [ ]:
# and our (ascending, Radarsat) LOS vector is...

p_asc

In [ ]:
# calculate the LOS vector components for the descending data (you need to enter the values)

dsc_azi=np.radians()

dsc_inc=np.radians()

p_dsc=np.array([np.cos(dsc_azi)*np.sin(dsc_inc),-np.sin(dsc_azi)*np.sin(dsc_inc),-np.cos(dsc_inc) ])

In [ ]:
# and our (descending, ERS) LOS vector is...

p_dsc

Do these values make sense? Things to look for:
- Are the relative sizes of the horizontal and vertical components consistent with the different incidence angles?
- Are the signs of the east components consistent with the look directions in the figures?
- Are the north components consistent with a polar-orbiting satellite (i.e. small)?

## 3. How to invert for decomposed LOS velocities

The next thing we need to do is to take these LOS vectors and set up an inversion so that we can decompose our LOS velocities into something a little more intuitive.

We have **two observations** (velocities from two different viewing geometries) and so we can solve for velocities in **two directions**. Given that InSAR is primarily sensitive to deformation in the **vertical** and **east-west** directions, it is common to decompose velocities into those components. 

Let's start with what we want to obtain. Let's define a vector, **m**, of decomposed velocities,

$${\bf m} = [m_H, m_V]^T$$

Here, $m_H$ is the horizontal (i.e. east) velocity, and $m_V$ is the vertical velocity.

We will use our two range change velocity observations, $r_{asc}$ and $r_{dsc}$ (for the ascending and descending data sets, respectively), to form a vector, **r** of observations:

$${\bf r} = [r_{asc}, r_{dsc}]^{T}$$

Next, we need to assemble the relevant parts of our LOS vectors into a *design matrix*, **P**. This is a matrix that relates the 'model velocities', **m**, to range-change velocities:

$${\bf r'} = {\bf P~m}$$

where **r'** here is a vector of forward-modeled range-change velocities that simulate the 'real' range-change velocities, **r**.

What goes into **P**? Well, we want to take the parts of the LOS vectors we want to decompose our LOS velocities into, like so:

$${\bf P} = \begin{pmatrix}p_{asc_H} & p_{asc_V} \\ p_{dsc_H} & p_{dsc_V} \end{pmatrix}$$  

where 

- $p_{asc_H} = p_{asc_x}$ is the east component of the ascending LOS vector
- $p_{asc_V} = p_{asc_z}$ is the vertical component of the ascending LOS vector
- $p_{dsc_H} = p_{dsc_x}$ is the east component of the descending LOS vector
- $p_{dsc_V} = p_{dsc_z}$ is the vertical component of the descending LOS vector

And once we have these things defined, we can do a bit of matrix math to solve for **m**:

$${\bf P~m} = {\bf r} $$ 
$${\bf P^TP~m} = {\bf P^Tr}$$
$${\bf(P^TP)}^{-1}{\bf P^TPm} = {\bf(P^TP)}^{-1}{\bf P^Tr}$$
$${\bf m} = {\bf(P^TP)}^{-1}{\bf P^Tr}$$

(This is the *least-squares* solution for **m** in fact.)

So now, we just need to extract the necessary parts of our LOS vectors to make **P** and the rest is easy!

In [ ]:
# take the east and north components from the ascending data...
p_hat_asc = np.array([p_asc[0], p_asc[2]])

# ...and the descending data
p_hat_dsc = np.array([p_dsc[0], p_dsc[2]])

# and finally, assemble all that into a matrix...
P=np.vstack(([p_hat_asc, p_hat_dsc]))

In [ ]:
# let's see what that looks like
P

## 4. Let's solve for some deformation rates!

So all that remains is to program up the solution, which is just some light numpy!

In [ ]:
# here are all the velocities for all the landslides
r_asc_all=np.array([-7.2,-4.3,-4.6,-2.8,-12.1])
r_dsc_all=np.array([12.2, 2.7, 5.3, 5.0, 12.0])

# and here is the inverse of P^TP which we will use several times
PTPinv=np.linalg.inv(np.matmul(P.T,P))

In [ ]:
# let's loop through them!
for i in range(len(r_asc_all)):
    
    # grab the values of range change rate for this landslide, make a vector
    r=np.vstack(([r_asc_all[i]],[r_dsc_all[i]]))
    
    # and solve for the decomposed velocities!
    u=np.matmul(PTPinv,np.matmul(P.T,r))

    # and let 'em know what you got
    print("landslide {0}".format(i+1))
    print("  east velocity={0:5.1f} mm/yr, vertical velocity={1:4.1f} mm/yr".format(u[0,0],u[1,0]))

Do these look plausible? The slides all seem to be moving in the downhill direction, which is to the west (negative east direction), and all but one have negative vertical directions (i.e. downward) $-$ and the one that has apparent uplift (landslide 2) could well be moving downward within error. 

## 5. Want to know more?

The descending track ERS data were published in a study by George Hilley and co-authors in 2004, which comes up with similar estimates of the average velocities by projecting velocities from each landslide into the downslope direction. They also noted that there was a surge in landslide movement during the El Nino winter of 1997-1998, a period of heavy rains.

Hilley, G. E., R. Burgmann, A. Ferretti, F. Novali and F. Rocca (2004), Dynamics of Slow-Moving Landslides from Permanent Scatterer Analysis, Science, 304, 1952$-$1955, https://doi.org/10.1126/science.1098821

For investigations into the efficacy of inverting for three-dimensional velocities (and why different incidences from the same flight/look direction do not provide enough information to solve for three components), this is a classic paper:

Wright, T. J., B. E. Parsons and Z. Lu (2004), Toward mapping surface deformation in three dimensions using InSAR, Geophys. Res. Lett., 31, L01607, https://doi.org/10.1029/2003GL018827.

See also a study of mine that builds off of the Wright et al. paper and actually succeeds in inverting for the three-dimensional deformation, by including dense azimuth offset information to constrain the north component:

Funning, G. J., B. Parsons, T. J. Wright, J. A. Jackson and E. J. Fielding. (2005), Surface displacements and source parameters of the 2003 Bam (Iran) earthquake from Envisat advanced synthetic aperture radar imagery, J. Geophys. Res., 110, B09406, https://doi.org/10.1029/2004JB003338.